# Biol 359A | Statistical Tests: ANOVA
### Spring 2024, Week 3
Objectives:
- Gain intuition for the relationship between sample size, effect size, and power
- Gain intuition for Type 1 error inflation and the Bonferroni correction
- Learn about 1-way ANOVAs and Tukey's test
- Learn about 2-way ANOVAs and interaction effects
- Interact with real data (stored as a matrix)

In [ ]:
# Import necessary libraries
from ipywidgets import interact, IntSlider, FloatSlider, Layout
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, f, f_oneway
from scipy import stats
import seaborn as sns
import statsmodels.stats.multicomp as mc
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison

In [ ]:
! rm -r week3_anova/
! git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr24/week3_anova.git
! cp -r week3_anova/* .
! ls

---

### Sample Size, Effect Size, and Power

Use the sliders in the vizualization below to explore the relationship. The code below will use the value specified by the "effect size" slider to generate data from two normal distributions with means `effect_size` apart. It then takes samples of the size specified by the "sample size" slider from each and performs a t-test. It repeats this sampling and t-test 200 times and then displays a histogram of the resulting p-values. The power of the test is displayed below the histogram.

In [ ]:

def simulate_t_test(sample_size, effect_size):
    # Generate fake data
    group1 = np.random.normal(0, 1, 10000)  # Control group with mean=0 and std=1
    group2 = np.random.normal(effect_size, 1, 10000)  # Experimental group with mean=effect_size and std=1
    
    p_values = []
    for _ in range(200):
        sample1 = np.random.choice(group1, sample_size)
        sample2 = np.random.choice(group2, sample_size)
        _, p_value = stats.ttest_ind(sample1, sample2)
        p_values.append(p_value)
    
    # Calculate power
    alpha = 0.05
    power = np.mean(np.array(p_values) < alpha)
    
    # Plot histogram of p-values
    plt.hist(p_values, bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('p-value')
    plt.ylabel('Frequency')
    plt.title('Histogram of p-values')
    plt.xlim(0, 1)  # Ensure x-axis does not go negative and is capped at 1
    plt.show()
    
    print(f"Power of the test (alpha = .05): {power:.3f}")

# Create interactive widgets
_ = interact(simulate_t_test,
         sample_size=IntSlider(min=10, max=1000, step=10, value=100, description='Sample Size'),
         effect_size=FloatSlider(min=0, max=1, step=0.1, value=0, description='Effect Size'))

ASSIGNMENT QUESTIONS:
- Please answer questions 11 and 12 in the assignment.

DISCUSSION QUESTIONS:
- Despite using a constant effect size in each simulation, the p-values vary widely across the 200 repetitions. Why is this? How does the effect size impact the distribution of p values? Why? 

---

### Repeated statistical tests and type 1 error

We have discussed several times how repeating statistical tests on the same data inflates type 1 error. The code below illustrates this point.

The code below generates fake gene expression data for 100 cells. Each cell has expression data for 100 genes. All gene expression data is pulled from the same uniform distribution. A random half of the cells are labeled "healthy" and the other half labeled "sick.

The code then performs a t-test comparing the mean expression of the healthy cells to the mean expression of the sick cells for all 40 genes (40 different t-tests). 

The plot on the left shows the distribution of all p-values for all 100 t-tests. The plot on the right shows the Cumulative Distribution Function (CDF) plot. Each point on the plot represents a p-value. The point's x value is the p-value and the point's y value is the percent of p-values in the set of all 100 that are equal to or less than that p-value. Under the null hypothesis (assuming no real difference between the healthy and sick cells' gene expression levels), we expect the p-values to follow a uniform distribution. This means that each p-value is equally likely, ranging from 0 to 1. On the CDF plot, a uniform distribution is represented by a diagonal line that runs from the bottom left to the top right of the graph. Deviations from this line might indicate departures from the expected distribution under the null hypothesis.

The number of significantly different genes found is reported below the plots along with the gene IDs and their respective p-values.

**The code below is non-deterministic**. This means that each time you run the code, the process will run with a new set of data and you will get different results. Run the cell a bunch of times to see a range of possible outcomes!

In [ ]:
# Generate random gene expression data
num_cells = 100  # Total number of cells, half healthy, half sick
num_genes = 100  # Number of genes per cell
gene_expression = np.random.rand(num_cells, num_genes) # uniform distribution [0, 1)
labels = np.array(["healthy"] * (num_cells // 2) + ["sick"] * (num_cells // 2))

# Perform t-tests and collect p-values
results = {"Gene": [], "P-Value": []}
for gene_idx in range(num_genes):
    healthy_expression = gene_expression[labels == "healthy", gene_idx]
    sick_expression = gene_expression[labels == "sick", gene_idx]
    _, p_value = ttest_ind(healthy_expression, sick_expression)
    results["Gene"].append(f"Gene {gene_idx + 1}")
    results["P-Value"].append(p_value)

# Set significance threshold
alpha = .05

# Convert results to DataFrame for easier handling
results_df = pd.DataFrame(results)
significant_results = results_df[results_df["P-Value"] < alpha]

# Plot histogram of p-values
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.hist(results_df["P-Value"], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('P-Value')
plt.ylabel('Frequency')
plt.title('Histogram of P-Values')

# Plot CDF of p-values
plt.subplot(1, 2, 2)
sorted_p_values = np.sort(results_df["P-Value"])
plt.plot(sorted_p_values, np.arange(1, len(sorted_p_values)+1) / len(sorted_p_values), marker='o', linestyle='none', color='blue')
plt.plot([0, 1], [0, 1], 'k--')  # Line of perfect uniformity
plt.xlabel('P-Value')
plt.ylabel('Cumulative Proportion')
plt.title('CDF of P-Values')

plt.tight_layout()
plt.show()

# Display results summary
print(f"Total tests conducted: {num_genes}")
print(f"Significant tests (P < 0.05): {len(significant_results)}")
print("\nSignificant tests details:")
print(significant_results)


DISCUSSION QUESTIONS:
- How many genes would we expect to be significantly different between the two populations? How does this compare to the number of genes you see are significantly different?
- How does the distribution of p-values compare to the distribution of p-values we would expect under the null hypothesis?
- Why do we expect the p-values to be uniformly distributed under the null hypothesis?

#### Bonferroni Correction
The code below repeats the process, but uses the Bonferroni correction. Remember, this means we divide $\alpha$ by the number of hypotheses being tested (in this case 100). Run this cell a bunch of times to see a range of possible outcomes.

In [ ]:
# Generate random gene expression data
num_cells = 100  # Total number of cells, half healthy, half sick
num_genes = 100  # Number of genes per cell
gene_expression = np.random.rand(num_cells, num_genes) # uniform distribution [0, 1)
labels = np.array(["healthy"] * (num_cells // 2) + ["sick"] * (num_cells // 2))

# Perform t-tests and collect p-values
results = {"Gene": [], "P-Value": []}
for gene_idx in range(num_genes):
    healthy_expression = gene_expression[labels == "healthy", gene_idx]
    sick_expression = gene_expression[labels == "sick", gene_idx]
    _, p_value = ttest_ind(healthy_expression, sick_expression)
    results["Gene"].append(f"Gene {gene_idx + 1}")
    results["P-Value"].append(p_value)

# Set significance threshold
alpha = .05
# Bonferroni correction
corrected_alpha = alpha / num_genes

# Convert results to DataFrame for easier handling
results_df = pd.DataFrame(results)
significant_results = results_df[results_df["P-Value"] < corrected_alpha]

# Plot histogram of p-values
plt.figure(figsize=(14, 6))
plt.subplot(1, 2, 1)
plt.hist(results_df["P-Value"], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('P-Value')
plt.ylabel('Frequency')
plt.title('Histogram of P-Values')

# Plot CDF of p-values
plt.subplot(1, 2, 2)
sorted_p_values = np.sort(results_df["P-Value"])
plt.plot(sorted_p_values, np.arange(1, len(sorted_p_values)+1) / len(sorted_p_values), marker='o', linestyle='none', color='blue')
plt.plot([0, 1], [0, 1], 'k--')  # Line of perfect uniformity
plt.xlabel('P-Value')
plt.ylabel('Cumulative Proportion')
plt.title('CDF of P-Values')

plt.tight_layout()
plt.show()

# Display results summary
print(f"Total tests conducted: {num_genes}")
print(f"Significant tests (P < {corrected_alpha}): {len(significant_results)}")
print("\nSignificant tests details:")
print(significant_results)


DISCUSSION QUESTIONS:
- How does the output of these tests compare to the outcomes of the uncorrected tests?

The Bonferroni correction is a powerful tool, but it comes at the cost of statistical power. The visualization below allows you to generate two datasets drawn from normal distributions with means `effect size` apart. It then repeats 100 t-tests on the same data. Explore how effect size and sample size affect statistical power before and after conducting a Bonferroni correction in a case where many statistical tests are performed on the same data.

In [ ]:
def simulate_t_test(sample_size, effect_size):
    # Parameters
    num_tests = 100  # Number of t-tests
    alpha = 0.05  # Significance level without correction
    corrected_alpha = alpha / num_tests  # Bonferroni corrected alpha
    
    # Generate fake data
    group1 = np.random.normal(0, 1, 10000)  # Control group with mean=0 and std=1
    group2 = np.random.normal(effect_size, 1, 10000)  # Experimental group with mean=effect_size and std=1
    
    p_values = []
    for _ in range(num_tests):
        sample1 = np.random.choice(group1, sample_size)
        sample2 = np.random.choice(group2, sample_size)
        _, p_value = stats.ttest_ind(sample1, sample2)
        p_values.append(p_value)
    
    # Calculate power
    power_before = np.mean(np.array(p_values) < alpha)
    power_after = np.mean(np.array(p_values) < corrected_alpha)
    
    # Plot histogram of p-values
    plt.hist(p_values, bins=20, color='skyblue', edgecolor='black')
    plt.xlabel('P-value')
    plt.ylabel('Frequency')
    plt.title('Histogram of P-values')
    plt.xlim(0, 1)  # Ensure x-axis does not go negative and is capped at 1
    plt.axvline(x=alpha, color='red', linestyle='--', label='Alpha (0.05)')
    plt.axvline(x=corrected_alpha, color='green', linestyle='--', label=f'Corrected Alpha ({corrected_alpha:.4f})')
    plt.legend()
    plt.show()
    
    # Print power of the test
    print(f"Power of the test before correction (alpha = .05): {power_before:.3f}")
    print(f"Power of the test after correction: {power_after:.3f}")

# Create interactive widgets
_ = interact(simulate_t_test,
         sample_size=IntSlider(min=10, max=1000, step=10, value=100, description='Sample Size'),
         effect_size=FloatSlider(min=0, max=1, step=0.1, value=0.1, description='Effect Size'))


DISCUSSION QUESTIONS:
- How does sample size affect statistical power?
- How do the statistical powers compare before and after the Bonferroni correction?

---

### The F Distribution

As we discussed in class, the test statistic in an ANOVA is the F statistic. Under the null hypothesis, the F statistic follows an F distribution. The shape of an F distribution varys depending on the number of data points you have and the number of groups you are comparing. Use the visualization below to explore how changes in the different degrees of freedom affect the shape of the distribution.

In [ ]:
def plot_f_distribution(df1, df2):
    # Define the range of x values
    x = np.linspace(0.01, 5, 1000)
    
    # Calculate the F-distribution
    y = f.pdf(x, df1, df2)
    
    # Plot the F-distribution
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, color='skyblue')
    plt.fill_between(x, y, color='skyblue', alpha=0.5)
    
    plt.title('F-distribution')
    plt.xlabel('F value')
    plt.ylabel('Probability density')
    plt.xlim(left=0)
    plt.ylim(bottom=0)
    plt.show()

# Custom layout to ensure slider descriptions are not cut off
custom_layout = Layout(width='800px', description_width='initial')

# Create interactive widgets with custom layout
_ = interact(plot_f_distribution,
         df1=IntSlider(min=1, max=30, step=1, value=5, description='Numerator Degrees of Freedom (DF1: Number of groups - 1)', style={'description_width': 'initial'}, layout=custom_layout),
         df2=IntSlider(min=1, max=100, step=1, value=20, description='Denominator Degrees of Freedom (DF2: Number of data points - Number of groups)', style={'description_width': 'initial'}, layout=custom_layout))


ASSIGNMENT QUESTIONS:
- Please complete assignment questions 13, 14 and 15

---

#### One-way ANOVA

One-way ANOVA (Analysis of Variance) is a statistical test used when we want to compare the means of more than two groups based on one independent variable. One-way ANOVA can help us understand if at least one group mean is different, but does not tell us which specific groups differ.

When to Use One-Way ANOVA:
- Multiple Groups Comparison: Use one-way ANOVA when you have three or more groups for comparison. For example, comparing the effectiveness of different teaching methods on student performance.
- One Independent Variable: There should be one categorical independent variable with two or more levels (e.g., treatment types).
- One Dependent Variable: The dependent variable should be continuous (e.g., test scores).
- Independence of Observations: Each group's observations should be independent of the others.
- Normality and Homogeneity of Variance: The data should approximately follow a normal distribution, and the variance among groups should be similar, though one-way ANOVA is fairly robust to deviations from these assumptions.

The following interactive visualization allows you to adjust the number of groups, the variance within those groups, and the mean difference between the groups. The code then generates 50 data points for each group and conducts a one-way ANOVA.

In [ ]:
def plot_one_way_anova(num_groups=3, within_group_var=1, mean_diff=0):
    np.random.seed(0)  # For reproducibility
    group_means = np.linspace(0, mean_diff, num=num_groups)
    data = [np.random.normal(loc=mean, scale=np.sqrt(within_group_var), size=50) for mean in group_means]
    
    # ANOVA
    f_stat, p_value = f_oneway(*data)
    
    # Plotting
    plt.figure(figsize=(12, 6))
    for i, group in enumerate(data):
        plt.scatter([i+1]*len(group), group, alpha=0.7, label=f'Group {i+1}')
    plt.boxplot(data, patch_artist=True, boxprops=dict(facecolor='lightblue'))
    plt.title(f'One-Way ANOVA: F-statistic = {f_stat:.2f}, p-value = {p_value:.3f}')
    plt.xlabel('Group')
    plt.ylabel('Values')
    plt.xticks(range(1, num_groups + 1))
    plt.legend()
    plt.grid(axis='y')
    plt.show()

# Adjusting widget layout
style = {'description_width': 'initial'}
# Custom layout to ensure slider descriptions are not cut off
custom_layout = Layout(width='400px', description_width='initial')
_ =interact(plot_one_way_anova,
         num_groups=widgets.IntSlider(min=2, max=5, step=1, value=3, description='Number of Groups', style=style, layout=custom_layout),
         within_group_var=widgets.FloatSlider(min=0.5, max=5, step=0.5, value=1, description='Within Group Variance', style=style, layout=custom_layout),
         mean_diff=widgets.FloatSlider(min=0, max=5, step=0.5, value=0, description='Mean Difference Between Groups', style=style, layout=custom_layout))


DISCUSSION QUESTIONS:
- We discussed in class how the F statistic is the ratio of the between group variance divided by the numerator degrees of freedom to the within group variance divided by the denominator degrees of freedom. In this way, the F statistic compares the between group variance to the within group variance. Does the between group variance need to be larger than the within group variance to get a significant P-value?

ASSIGNMENT QUESTIONS:
- Please complete question 16 in the assignment.

#### Tukey's HSD post-hoc test

Once we do a one-way ANOVA, even if we reject the null hypothesis we do not know which group means differ from which other group means. To determine this we need to do post-hoc analysis. We have already talked about the Bonferroni correction (which can be used after conducting and ANOVA to identify which groups have differences in means). Another very common post-hoc method is Tukey's Honestly Significant Difference (HSD) test. This test compares all possible pairs of means and sacrifices less power than the Bonferroni correction.

When to use Tukey's HSD post-hoc test:
- A one-way ANOVA has indicated significant differences among group means.
- There is a need to identify which specific pairs of groups differ.
- Maintaining an overall confidence level across a family of comparisons is important.

For this class, you won't need to understand specifically how a Tukey's test works. If you are interested, here are the steps involved (feel free to skip over numbered list below if not interested):
1. Calculate the standard error of the differences: Tukey's method calculates the standard error of the differences between means. This accounts for both the variability within groups and the sample size of each group.
2. Compute the range statistic: The range is the difference between the highest and lowest means among the groups being compared.
3. Adjust the range statistic: The raw range statistic is adjusted based on the sample size and the number of groups being compared.
4. Studentize the adjusted range: Studentizing involves dividing the adjusted range by your computed standard error.
5. Obtain critical values from the Studentized range distribution: Instead of dividing the alpha level by the number of comparisons like Bonferroni correction, Tukey's method uses the Studentized range distribution to obtain a critical value that depends on the alpha, the number of groups, and the total number of observations. These critical values help determine whether the observed differences between means are statistically significant.
6. Compare the differences between means to the critical value: Tukey's method compares the differences between means to the critical value obtained from the Studentized range distribution. If the absolute difference between two means is greater than the critical value times the standard error of the differences, then the difference is considered statistically significant.

The code below creates the same one-way ANOVA visualization as above but performs a Tukey's post-hoc test after to test for differences in significance of means.


In [ ]:
def plot_one_way_anova(num_groups=3, within_group_var=1, mean_diff=0):
    np.random.seed(0)  # For reproducibility
    group_means = np.linspace(0, mean_diff, num=num_groups)
    data = [np.random.normal(loc=mean, scale=np.sqrt(within_group_var), size=50) for mean in group_means]
    
    # ANOVA
    f_stat, p_value = f_oneway(*data)
    
    # Plotting
    plt.figure(figsize=(12, 6))
    for i, group in enumerate(data):
        plt.scatter([i+1]*len(group), group, alpha=0.7, label=f'Group {i+1}')
    plt.boxplot(data, patch_artist=True, boxprops=dict(facecolor='lightblue'))
    plt.title(f'One-Way ANOVA: F-statistic = {f_stat:.2f}, p-value = {p_value:.3f}')
    plt.xlabel('Group')
    plt.ylabel('Values')
    plt.xticks(range(1, num_groups + 1))
    plt.legend()
    plt.grid(axis='y')
    plt.show()

    # Preparing data for Tukey's HSD
    all_data = np.concatenate(data)
    groups = np.array([])
    for i in range(num_groups):
        groups = np.append(groups, [f"Group {i+1}"] * 50)
    
    # Tukey's HSD test
    tukey = mc.pairwise_tukeyhsd(endog=all_data, groups=groups, alpha=0.05)
    print(tukey)

# Adjusting widget layout
style = {'description_width': 'initial'}
custom_layout = Layout(width='400px', description_width='initial')
_ = interact(plot_one_way_anova,
         num_groups=IntSlider(min=2, max=5, step=1, value=3, description='Number of Groups', style=style, layout=custom_layout),
         within_group_var=FloatSlider(min=0.5, max=5, step=0.5, value=1, description='Within Group Variance', style=style, layout=custom_layout),
         mean_diff=FloatSlider(min=0, max=5, step=0.5, value=0, description='Mean Difference Between Groups', style=style, layout=custom_layout))


In the Tukey's HSD table:
- the first two columns indicate the group means being compared (ex the first row above compares the means of group 1 and 2, the second row compares group 1 and group 3)
- the meandiff column lists the difference between the means of these groups
- the p-adj column is the p value of the comparison between means
- the lower and upper columns define the lower and upper bounds of the confidence interval, or the minimum and maximum differences in mean that would be considered non-significant when comparing the two provided means.
- the reject column says whether or not we reject the null hypothesis that the means are equal for each comparison

ASSIGNMENT QUESTIONS:
- Please complete questions 18 and 19 in the assignment (we will come back to 17 don't worry!)

#### Two-way ANOVA
Two-way ANOVA (Analysis of Variance) extends the principles of one-way ANOVA to examine the effects of two independent variables (factors) on a dependent variable simultaneously. This method not only assesses the main effect of each factor but also investigates whether there is an interaction between them. This interaction effect is crucial, as it indicates whether the effect of one factor depends on the level of the other factor.

When to Use Two-Way ANOVA:
- Multiple Factors: Use two-way ANOVA when your study involves two independent variables and you are interested in understanding both their individual (main effects) and combined (interaction effect) impacts on a dependent variable.
- Interaction Effects: Two-way ANOVA is particularly valuable when you suspect that the impact of one independent variable on the dependent variable might change across levels of the other independent variable.
 - Example Contexts: Comparing student performance across different teaching methods (Factor 1) and class sizes (Factor 2) to see not only which method and size are best but also if some methods work better or worse in different sizes.

This is a bit abstract, so lets ground it in an example.

Imagine a study designed to investigate the growth of a certain plant species. The study examines the effects of two factors:

- Factor 1 (Fertilizer Type): Two levels - Organic (O) and Synthetic (S).
- Factor 2 (Watering Frequency): Two levels - Frequent (F) and Infrequent (I).

The dependent variable is the plant growth measured in centimeters after a certain period.

- Main Effect 1 corresponds to the type of fertilizer used. It measures the average difference in plant growth between plants fertilized with Organic and those with Synthetic fertilizers, disregarding the watering frequency.
- Main Effect 2 corresponds to the watering frequency. It measures the average difference in plant growth between plants watered frequently and those watered infrequently, regardless of the fertilizer type.
- Interaction Effect examines whether the impact of the fertilizer type on plant growth depends on the watering frequency. It's possible, for instance, that Organic fertilizer leads to better growth with Frequent watering but not as much with Infrequent watering, suggesting the effectiveness of a fertilizer type can be influenced by how often the plants are watered.

The following interactive visualization allows you to visualize the main and interaction effects of the two factors on a dependent variable. The ANOVA table output is displayed below the plot.

In [ ]:
def simulate_two_way_anova(main_effect1=2.0, main_effect2=0.0, interaction_effect=0.0):
    np.random.seed(42)
    num_samples = 30
    baseline_growth = 5

    # Factors
    fertilizer = np.repeat(["Organic", "Synthetic"], num_samples * 2)
    watering = np.tile(np.repeat(["Infrequent", "Frequent"], num_samples), 2)
    
    # Calculate mean growth for each combination
    mean_growth = np.concatenate([
        np.random.normal(baseline_growth, 0.5, num_samples),  # Organic, Infrequent
        np.random.normal(baseline_growth + main_effect2, 0.5, num_samples),  # Organic, Frequent
        np.random.normal(baseline_growth + main_effect1, 0.5, num_samples),  # Synthetic, Infrequent
        np.random.normal(baseline_growth + main_effect1 + main_effect2 + interaction_effect, 0.5, num_samples)  # Synthetic, Frequent
    ])
    
    # Prepare DataFrame for two-way ANOVA
    df = pd.DataFrame({
        "Growth": mean_growth,
        "Fertilizer": fertilizer,
        "Watering": watering
    })
    
    # Two-way ANOVA
    model = ols('Growth ~ C(Fertilizer) + C(Watering) + C(Fertilizer):C(Watering)', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    # Plotting
    labels = ['Organic + Infrequent', 'Organic + Frequent', 'Synthetic + Infrequent', 'Synthetic + Frequent']
    means = df.groupby(["Fertilizer", "Watering"]).mean().squeeze()
    stds = df.groupby(["Fertilizer", "Watering"]).std().squeeze()
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(labels, means, yerr=stds, capsize=5, color=['lightblue', 'lightgreen', 'salmon', 'gold'], alpha=0.7)
    plt.title('Plant Growth Study: Mean Growth with Different Treatments')
    plt.ylabel('Mean Growth (cm)')
    plt.xlabel('Treatments (Fertilizer Type + Watering Frequency)')
    for bar, mean in zip(bars, means):
        plt.text(bar.get_x() + bar.get_width() / 2, mean, f'{mean:.2f}', ha='center', va='bottom')
    plt.show()
    
    # Display ANOVA table
    print(anova_table)

# Interactive widgets with adjusted layout for slider descriptions
style = {'description_width': 'initial'}
custom_layout = Layout(width='450px')
_ = interact(simulate_two_way_anova,
             main_effect1=FloatSlider(min=0, max=5, step=0.1, value=2, description='Effect of Synthetic Fertilizer', style=style, layout=custom_layout),
             main_effect2=FloatSlider(min=0, max=5, step=0.1, value=0, description='Effect of Frequent Watering', style=style, layout=custom_layout),
             interaction_effect=FloatSlider(min=-2, max=2, step=0.1, value=0.0, description='Interaction Effect', style=style, layout=custom_layout))


In the ANOVA table:
- the left-most column lists the factors impacting your data. These include one for each independent variable that has varied in your data and one for each interaction between independent variables in your data. With two factors there is only one interaction. This also includes a row for the Residual, or the amount of variation in the data that is not explained by any independent variables. No F statistic or P value is calculated for the Residual.
- the sum_sq column lists the sum of squared differences between each group and the overall data mean
- the df column lists the degrees of freedom associated with each source of variation. This value is (number_of_factors - 1)
- the F column lists the F statistics for each factor
- the PR(>F) column lists the corresponding p values

DISCUSSION QUESTIONS:
- What does interaction effect mean?
- If your ANOVA tells you there is a significant effect of one independent variable on your dependent variable, do you know which group means are different from which other group means?

ASSIGNMENT QUESTIONS:
- Please answer question 17 in the assignment

---

### Data as a Matrix

The code below imports and cleans some blood test data:

In [ ]:
def clean_lab_data(n=12):
    lab_data = pd.read_csv("data/hcvdat0.csv", index_col=0)
    lab_data = lab_data[lab_data["Category"] != "0s=suspect Blood Donor"]
    lab_data = lab_data.dropna(axis=0)
    df_list = []
    for category, category_df in lab_data.groupby("Category"):
        df_list.append(category_df.sample(n, random_state=5))
    
    return pd.concat(df_list)
        

lab_data = clean_lab_data()
print(lab_data.shape)
lab_data

- albumin (ALB)

- alkaline phosphatase (ALP)

- alanine amino-transferase (ALT)

- aspartate amino-transferase (AST)

- bilirubin (BIL) 

- choline esterase (CHE)

- cholesterol (CHOL)

- creatinine (CREA)* 

- γ-glutamyl-transferase (GGT)

- total protein (PROT)*

\* I think, the dataset was not very well annotated.

Source: https://archive.ics.uci.edu/ml/datasets/HCV+data#

We have gone through and cleaned out some of the data that was incomplete, and also reduced the samples to be the same across all groups.

ASSIGNMENT QUESTION:
- Please answer question 20 in the assignment

---

#### ANOVA in practice

Do the values for any of the measured features in the blood tests differ between different disease states? Lets go through the steps involved in an actual ANOVA to find out.

**Exploratory Data Analysis**: Before we perform ANOVA, we should do some initial exploratory data analysis to understand the data distribution and to check the assumptions for ANOVA.

First it is always a good idea to visually inspect your data. The plots below show the distributions of each feature for each disease state. Take a look at the plots to see if any features seem to have an abundance of outliers or very different variances between groups (warning this code makes a lot of plots!)

In [ ]:
# Plotting distributions of continuous variables for each category
for column in lab_data.select_dtypes(include=['float64', 'int']).columns:
    sns.boxplot(x='Category', y=column, data=lab_data)
    plt.title(f'Distribution of {column} by Category')
    plt.show()
    
    # Histogram for each category
    g = sns.FacetGrid(lab_data, col="Category", margin_titles=True, height=4)
    g.map(plt.hist, column, color="steelblue")
    plt.show()


**Testing for Normality**: 
The Shapiro-Wilk test can assess the normality of the distributions of each variable. The code below runs the Shapiro-wilk test for each variable.

In [ ]:
# List of continuous variable names in DataFrame
continuous_variables = lab_data.select_dtypes(include=['float', 'int']).columns.tolist()

# Define significance level
alpha = 0.05

# Function to perform Shapiro-Wilk Test for Normality and report failures
def shapiro_wilk_test(data, variable, alpha=0.05):
    print(f"\nShapiro-Wilk Test for Normality - {variable}:")
    failed_categories = []
    for category in data['Category'].unique():
        _, p_value = stats.shapiro(data[data['Category'] == category][variable].dropna())
        if p_value < alpha:
            failed_categories.append(category)
        print(f"Category {category}: p-value = {p_value}")
    if failed_categories:
        print(f"-> Normality assumption may be violated for {variable} in categories: {', '.join(failed_categories)}")
    else:
        print(f"-> All categories passed normality tests for {variable}")

# Running the tests for each continuous variable
for variable in continuous_variables:
    shapiro_wilk_test(lab_data, variable, alpha)

View the above as a scrollable element to see which distributions failed the normality test. There are data transformations we can do to make data more normally distributed, but for now lets just remove all of the features that fail the Shapiro-Wilk Test for Normality

In [ ]:
normal_lab_data = lab_data.drop(columns=['PROT', 'GGT', 'CREA', 'CHOL', 'BIL', 'AST', 'ALT', 'ALP'])
normal_lab_data

**The Levene Test for Homogeneity of Variances** The Levene test assesses the homogeneity of variances. The code below runs the Levene test on all the normally distributed continuous variables from our original dataset.

In [ ]:
# List of continuous normally distributed variable names
continuous_variables = normal_lab_data.select_dtypes(include=['float', 'int']).columns.tolist()

# Define significance level
alpha = 0.05

# Function to perform Levene's Test for Homogeneity of Variances and report failure
def levene_test(data, variable, alpha=0.05):
    samples = [data[data['Category'] == category][variable].dropna() for category in data['Category'].unique()]
    stat, p_value = stats.levene(*samples)
    print(f"\nLevene's Test for Homogeneity of Variances - {variable}: p-value = {p_value}")
    if p_value < alpha:
        print(f"-> Homogeneity of variances assumption may be violated for {variable}")
    else:
        print(f"-> Homogeneity of variances assumption not violated for {variable}")

# Running the tests for each continuous variable
for variable in continuous_variables:
    levene_test(lab_data, variable, alpha)

Great! We have no violoations of Homogeneity of variance among these normally distributed variables! We can go ahead with our ANOVA!

#### One-Way ANOVA Test

The code below allows you to select one of these three variables and perform a one-way ANOVA to see if the mean values for each variable differ between disease states.

In [ ]:
# List of continuous variable names in your DataFrame
continuous_variables = normal_lab_data.select_dtypes(include=['float', 'int']).columns.tolist()

def perform_anova(variable):
    # Performing ANOVA
    model = ols(f'{variable} ~ C(Category)', data=lab_data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    # Printing ANOVA table
    print(anova_table)
    
    # Visualizing the distributions
    sns.boxplot(x='Category', y=variable, data=lab_data)
    plt.title(f'Distribution of {variable} by Category')
    plt.show()

# Creating dropdown for variable selection
variable_dropdown = Dropdown(options=continuous_variables, description='Variable:')

# Linking the dropdown to the ANOVA function
interact(perform_anova, variable=variable_dropdown)

Wow! There are differences between means for each of the variables! However, we don't know which means are different! To do this we need to do a post-hoc test.

#### Post-Hoc test

The code below allows you to perform a Tukey's post-hoc test and a Bonferroni corrected pairwise comparison between all means for a variable of your choice. Which categories have differences in means for which categories?

In [ ]:
def perform_post_hoc_tests(variable):
    # Performing Tukey's HSD test
    tukey = pairwise_tukeyhsd(endog=lab_data[variable],
                              groups=lab_data['Category'],
                              alpha=0.05)
    print("Tukey's HSD Test Result:")
    print(tukey.summary())
    
    # Bonferroni corrected pairwise comparisons
    print("\nBonferroni Corrected Pairwise Comparisons:")
    mc = MultiComparison(lab_data[variable], lab_data['Category'])
    bonf_result = mc.allpairtest(stats.ttest_ind, method='bonf')
    print(bonf_result[0])
    
    # Optionally, plot Tukey's HSD test result
    #tukey.plot_simultaneous(xlabel=variable, ylabel='Group')
    #plt.title('Tukey HSD Test Result')
    #plt.show()

# Creating dropdown for variable selection
variable_dropdown = Dropdown(options=continuous_variables, description='Variable:')

# Linking the dropdown to the post hoc tests function
interact(perform_post_hoc_tests, variable=variable_dropdown)

DISCUSSION QUESTION:
- Why are the Tukey's post-hoc analysis findings the same as the Bonferroni post-hoc analysis findings?